In [47]:
from typing import Tuple
import torch

def gd_factorise_ad(A: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    U = torch.rand(list(A.size())[0], rank, requires_grad = True)
    V = torch.rand(list(A.size())[1], rank, requires_grad = True)
    
    for epoch in range(0, num_epochs):
        loss = torch.nn.functional.mse_loss(A, U@V.t() , reduction='sum')
        loss.backward()
        
        U.data = U.data - lr*U.grad
        V.data = V.data - lr*V.grad
        
        U.grad.data.zero_()
        V.grad.data.zero_()

    return U,V

A = torch.tensor([[0.3374,0.6005,0.1735] , [3.3359,0.0492,1.8374] , [2.9407,0.5301,2.2620]], requires_grad = True)
U_cap , V_cap = gd_factorise_ad(A , 2)


recon_loss = torch.nn.functional.mse_loss(A, U_cap@V_cap.t(), reduction='sum')
print("The reconstruction error is : " , recon_loss)

The reconstruction error is :  tensor(0.1219, grad_fn=<MseLossBackward0>)


In [48]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases'+'/iris/iris.data', header=None)
data = torch.tensor(df.iloc[:, [0,1,2,3]].values) 
data = data - data.mean(dim=0)

new_U, new_V = gd_factorise_ad(data, 2)
data_recon_loss = torch.nn.functional.mse_loss(data, new_U@new_V.t(), reduction='sum')
print("The reconstruction error is : " , data_recon_loss.item())


RuntimeError: Found dtype Double but expected Float

In [51]:
u , s , v = torch.svd(data)

fn_data = torch.mm(torch.mm(u, torch.diag(s)), v)
new_recon_loss = torch.nn.functional.mse_loss(data, fn_data , reduction='sum')
print("Reconstruction loss before setting last singular value to zero : " , new_recon_loss)

Reconstruction loss before setting last singular value to zero :  tensor(304.1669, dtype=torch.float64)


In [60]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases'+'/iris/iris.data', header=None)
df = df.sample(frac=1) #shuffle

mapping = {k: v for v, k in enumerate(df[4].unique())} 
df[5] = df[4].map(mapping)

alldata = torch.tensor(df.iloc[:, [0,1,2,3]].values, dtype=torch.float) 
alldata = (alldata - alldata.mean(dim=0)) / alldata.var(dim=0)

targets_tr = torch.tensor(df.iloc[:100, 5].values, dtype=torch.long) # 100 x 1
targets_va = torch.tensor(df.iloc[100:, 5].values, dtype=torch.long) # 50 x 1
data_tr = alldata [:100] # 100 x 4
data_va = alldata [100:] # 50 x 4
    
    
def train_function(X_train, y_train, epochs=100, lr = 0.01):
    w1 = torch.rand(4, 12, requires_grad = True) # Hard coding this becuase the matrix dimensions are given in the spec sheet
    w2 = torch.rand(12, 3, requires_grad = True) # Hard coding this becuase the matrix dimensions are given in the spec sheet
    b1 = torch.tensor(0.0, requires_grad = True)
    b2 = torch.tensor(0.0, requires_grad = True)
    
    for epoch in range(epochs):
    
        logits = torch.relu(X_train @ w1 + b1) @ w2 + b2
        loss_value = torch.nn.functional.cross_entropy(logits, y_train)
    
        loss_value.backward()
    
        w1.data = w1.data - lr*w1.grad
        w2.data = w2.data - lr*w2.grad
        b1.data = b1.data - lr*b1.grad
        b2.data = b2.data - lr*b2.grad
        
    return w1, w2, b1, b2


def test_function(X_test, y_test, w1, w2, b1, b2):
    logits = torch.relu(X_test @ w1 + b1) @ w2 + b2
    loss_value = torch.nn.functional.cross_entropy(logits, y_test)
    return loss_value

w1, w2, b1, b2 = train_function(data_tr, targets_tr)
validation_accuracy = test_function(data_va, targets_va, w1, w2, b1, b2)

print("The validation loss is: " , validation_accuracy.item())

The validation loss is:  tensor(0.8722, grad_fn=<NllLossBackward0>)


In [61]:
# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
loss.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
loss = F.cross_entropy(input, target)
loss.backward()

NameError: name 'F' is not defined

In [ ]:
# Doubts : 
    # 1) In the first cell, the formula is changed from + to -. Is it how it is supposed to be done?